<a href="https://colab.research.google.com/github/prasadmahamulkar/Large-Language-Models/blob/main/RAG_Mixtral_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG - Mixtral Model
📒Notebook Created by ❤️ [@prasadmahamulkar](https://www.linkedin.com/in/prasad-mahamulkar/).



In [ ]:
!pip install -q -U transformers langchain sentence_transformers faiss-gpu huggingface-hub python-dotenv PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.2 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

text = get_pdf_text(["/content/qlora_paper.pdf"])

text[:100]

'QL ORA: Efficient Finetuning of Quantized LLMs\nTim Dettmers∗Artidoro Pagnoni∗Ari Holtzman\nLuke Zettl'

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks
chunk_text = get_text_chunks(text)
chunk_text[0]

'QL ORA: Efficient Finetuning of Quantized LLMs\nTim Dettmers∗Artidoro Pagnoni∗Ari Holtzman\nLuke Zettlemoyer\nUniversity of Washington\n{dettmers,artidoro,ahai,lsz}@cs.washington.edu\nAbstract\nWe present QLORA, an efficient finetuning approach that reduces memory us-\nage enough to finetune a 65B parameter model on a single 48GB GPU while\npreserving full 16-bit finetuning task performance. QLORAbackpropagates gradi-\nents through a frozen, 4-bit quantized pretrained language model into Low Rank\nAdapters (LoRA). Our best model family, which we name Guanaco , outperforms\nall previous openly released models on the Vicuna benchmark, reaching 99.3%\nof the performance level of ChatGPT while only requiring 24 hours of finetuning\non a single GPU. QLORAintroduces a number of innovations to save memory\nwithout sacrificing performance: (a) 4-bit NormalFloat (NF4), a new data type that\nis information theoretically optimal for normally distributed weights (b) Double'

In [ ]:
model = "BAAI/bge-small-en-v1.5"
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceEmbeddings(model_name=model, encode_kwargs=encode_kwargs, model_kwargs={"device": "cpu"})

In [ ]:
check = embeddings.embed_query("what is Qlora?")
check[0:10]

[-0.10743825137615204,
 0.057859934866428375,
 -0.11160264164209366,
 0.04787628725171089,
 -0.10750839114189148,
 0.010762185789644718,
 0.07565398514270782,
 0.028037123382091522,
 0.09083810448646545,
 0.0040848273783922195]

In [ ]:
vectorstore = FAISS.from_texts(texts=chunk_text, embedding=embeddings)

retriever = vectorstore.as_retriever(score_threshold = 0.9)

In [ ]:
result = retriever.get_relevant_documents("what is Qlora?")
result

[Document(page_content='large corporations and small teams with consumer GPUs.\nAnother potential source of impact is deployment to mobile phones. We believe our QLORAmethod\nmight enable the critical milestone of enabling the finetuning of LLMs on phones and other low\nresource settings. While 7B models were shown to be able to be run on phones before, QLORAis\nthe first method that would enable the finetuning of such models. We estimate that with an iPhone 12\nPlus, QLORAcan finetune 3 million tokens per night while the phone is charging. While finetuned\n7B models do not reach the quality of ChatGPT, we believe that the quality is good enough to enable\nnovel applications that have not been possible before due to privacy or LLM quality issues. QLORA\ncan help enable privacy-preserving usage of LLMs, where users can own and manage their own data\nand models, while simultaneously making LLMs easier to deploy.\nHowever, finetuning is a dual-use technology that can be abused to cause ha

In [ ]:
llm = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
                     model_kwargs={"temperature": 0.1, "max_length": 1048},
                huggingfacehub_api_token="YOUR_HUGGINGFACEHUB_API_TOKEN")

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm, retriever=vectorstore.as_retriever(), memory=memory
    )

In [ ]:
conversation_chain({"question": "What is Qlora?"}, return_only_outputs=True)

{'answer': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nlarge corporations and small teams with consumer GPUs.\nAnother potential source of impact is deployment to mobile phones. We believe our QLORAmethod\nmight enable the critical milestone of enabling the finetuning of LLMs on phones and other low\nresource settings. While 7B models were shown to be able to be run on phones before, QLORAis\nthe first method that would enable the finetuning of such models. We estimate that with an iPhone 12\nPlus, QLORAcan finetune 3 million tokens per night while the phone is charging. While finetuned\n7B models do not reach the quality of ChatGPT, we believe that the quality is good enough to enable\nnovel applications that have not been possible before due to privacy or LLM quality issues. QLORA\ncan help enable privacy-preserving usage of LLMs, where users can own and manage t

# If you want understand what exactly the model is doing, you can use this code:
prompt is not good but you get the idea

In [ ]:
from langchain.prompts import PromptTemplate

template = """
<s>[INST] <<SYS>>
Act as a Qlora Expert. Use the following information to answer the question at the end.
<</SYS>>

{context}

{question} [/INST]
"""


PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
output_query = chain('What is Qlora?')
print(output_query["result"].strip())


<s>[INST] <<SYS>>
Act as a Qlora Expert. Use the following information to answer the question at the end.
<</SYS>>
 
large corporations and small teams with consumer GPUs.
Another potential source of impact is deployment to mobile phones. We believe our QLORAmethod
might enable the critical milestone of enabling the finetuning of LLMs on phones and other low
resource settings. While 7B models were shown to be able to be run on phones before, QLORAis
the first method that would enable the finetuning of such models. We estimate that with an iPhone 12
Plus, QLORAcan finetune 3 million tokens per night while the phone is charging. While finetuned
7B models do not reach the quality of ChatGPT, we believe that the quality is good enough to enable
novel applications that have not been possible before due to privacy or LLM quality issues. QLORA
can help enable privacy-preserving usage of LLMs, where users can own and manage their own data
and models, while simultaneously making LLMs easier to 